In [1]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output
import numpy as np

from tensorflow.keras.utils import to_categorical

list_of_lists = []
with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        list_of_lists.append(inner_list)
print(len(list_of_lists[0])) # About 4 MB
ecoli = list_of_lists[0]


4638690


In [2]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [3]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0).fit(X, y)
gpc.score(X, y)

gpc.predict_proba(X[:2,:])

array([[0.83548752, 0.03228706, 0.13222543],
       [0.79064206, 0.06525643, 0.14410151]])

In [4]:
s = ecoli
char_list = [97, 103, 99, 116] # we can read this as we go
print(char_list)
update_period = len(s)

k = 64 #context length
n = 10000 # train every
legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.
vocab_size = len(char_list)

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
#Train model
x = np.zeros((update_period-k, k)) # 64 characters context
y = np.zeros((update_period-k))

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0)
print(len(s))
idx3 = 0
for idx2 in range(k,len(s)):
    train_seq = [legend[i] for i in s[idx2-k:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq)
    y[idx3] = train_target
    idx3 += 1

[97, 103, 99, 116]
4638690


## GP explodes. Not a great plan.

In [5]:
gpc.fit(x[:100000],y[:100000])

MemoryError: Unable to allocate 74.5 GiB for an array with shape (100000, 100000) and data type float64

In [ ]:
predicted_onehot = list(gpc.predict_proba(x[100000:200000]))

In [ ]:
cc = ['a','g','c','t']
predicted_label = [cc[np.argmax(i)] for i in predicted_onehot]

print(np.sum(np.array(predicted_label) == np.array(ecoli[100000+k:200000+k])))